<a href="https://colab.research.google.com/github/shstreuber/Data-Mining/blob/master/Module12_Transformers_Chatbot.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#**0. What Is A Chatbot?**

<center>
<img src = "https://github.com/shstreuber/Data-Mining/blob/master/images/chatbot_meme.JPG?raw=true" height=400>
</center>

AI Chatbots are applications that businesses and other organizations use to automate conversations between AI and humans. These conversations can occur through text (as with Copilot) or speech (as with Alexa, for example). Chatbots must comprehend and imitate human conversation when engaging with users.

In order to process a large amount of natural language data, an AI uses NLP or Natural Language Processing. NLP tasks involve breaking down human text and audio signals from voice data in ways that computers can analyze and convert into comprehensible data. Some of the tasks in NLP data ingestion include:

1. **Speech Recognition,** which involves converting speech into text using a subprocess called speech tagging, which allows a computer to break down speech and add context, accents, or other speech attributes.
2. **Word Sense Disambiguation,** which selects the most appropriate meaning for a word based on its context. For instance, it helps determine whether a word functions as a verb or a pronoun.
3. **Named Entity Recognition (NER),** Nwhic identifies words and phrases as specific entities, such as recognizing “Dav” as a person’s name or “America” as thase name of a country.
4. **Sentiment Analysis,** which focuses on extracting nuances and hidden emotions, like attitude, sarcasm, fear, or joy, from human speech.

There are essentially two types of chatbots:
1. **Rule-Based (Scripted) Chatbots,** which operate based on predetermined scripts stored in their library. When a user inputs a query, the chatbot replies according to the predefined script within its library. One drawback of this type of chattbot is that users must structure their queries very precisely.
2. **Self-Learning (Artificially Intelligent) Chatbots,** which rely on a combination of NLP (for analysis) and AI in order to respond to nuanced questions and learn from each interaction to provide improved responses in the future.

And then a combination of the two.

##**Using Transformer Neural Networks to build a Chatbot**

Now we are talking about the "T" from ChatGPT, created in the research labs of Google, in 2017. In short: A transformer is a deep learning model that adopts the mechanism of self-attention, differentially weighting the significance of each part of the input (which includes the recursive output!) data.

###**What is a Transformer Network?**

The transformer neural network is a novel architecture that aims to solve sequence-to-sequence tasks while handling long-range dependencies with ease. It was first proposed in the paper **[“Attention Is All You Need.”](https://arxiv.org/abs/1508.04025)** and is now a state-of-the-art technique in the field of NLP.

All RNNs struggle in some form or another when dealing with long sentences, either with long training times, high processing load, or vanishing gradients etc. So, a solution came along in a paper that introduced attention. It highly improved the quality of machine translation as it allows the model to focus on the relevant part of the input sequence as necessary. This attention model is different from the classic sequence-to-sequence model in two ways:

1. First, as compared to a simple seq-to-seq model, here, the encoder passes a lot more data to the decoder. Previously, only the final, hidden state of the encoding part was sent to the decoder, but now the encoder passes all the hidden states, even the intermediate ones.
2. The decoder part also does an extra step before producing its output. This step proceeds like this:
 1. It checks each hidden state that it received as every hidden state of the encoder is mostly associated with a particular word of the input sentence.
 2. It gives each hidden state a score.
 3. Each score is multiplied by its respective softmax score, thus amplifying hidden states with high scores and drowning out hidden states with low scores.

<center>
<img src = "https://github.com/shstreuber/Data-Mining/blob/master/images/transformer_overview.JPG?raw=true">
</center>

The graphic above comes from [this paper](https://www.researchgate.net/publication/369476958_TRON_Transformer_Neural_Network_Acceleration_with_Non-Coherent_Silicon_Photonics).

**EVEN BETTER:**

The dynamic visualization below shows you how this works. The entire **AWESOME** explanation is available [here](https://jalammar.github.io/visualizing-neural-machine-translation-mechanics-of-seq2seq-models-with-attention/).

In [1]:
from IPython.display import IFrame  # This is just for me so I can embed videos
IFrame(src="https://jalammar.github.io/images/attention_tensor_dance.mp4", width=560, height=315)

Here is how the attention process works:
1. The attention decoder RNN takes in the embedding of the <END> token, and an initial decoder hidden state.
2. The RNN processes its inputs, producing an output and a new hidden state vector (h4). The output is discarded.
3. **Attention Step:** We use the encoder hidden states and the h4 vector to calculate a context vector (C4) for this time step.
4. We concatenate h4 and C4 into one vector.
5. We pass this vector through a feedforward neural network (one trained jointly with the model).
6. The output of the feedforward neural networks indicates the output word of this time step.
7. Repeat for the next time steps

The video below gives you a **GREAT SUMMARY** of everything you have just learned:

In [2]:
from IPython.display import IFrame  # This is just for me so I can embed videos
IFrame(src="https://www.youtube.com/embed/zxQyTK8quyY", width=560, height=315)

Now let's start building our own Chatbots **THREE** different ways:

#**1. Basic Text Input Chatbot**

Read [this article on Analytics Vidhya](https://www.analyticsvidhya.com/blog/2021/10/complete-guide-to-build-your-ai-chatbot-with-nlp-in-python/). Here, the author is building a speech-to-text chatbot natively on Microsoft Windows. To get the code from this post working in Google Colab, we have to make some adjustments since there's an issue with building the PyAudio wheel in Google Colab. That is because, unfortunately, Google Colab does not support building packages that require compiling C extensions, such as PyAudio. That works only in MS Windows. I learned that the hard way.

To rewrite the code so that it runs in Google Colab without requiring PyAudio or needing to connect to your local microphone, we will replace the speech-to-text functionality with text input.

**NOTE:** The code below  generates a right-padding warning (which sent me down a fruitless hours-long debugging path) but works as intended by carrying on a conversation.

First, we import all the libraries we need

In [3]:
import transformers
import os
import time
import datetime
import numpy as np

Next, we build a ChatBot class to provide functionality for initializing the chatbot, getting user input, responding to user input, checking for specific commands, and providing the current time.:

1. Initialization (__init__ method):
When you create a new instance of the ChatBot class, it initializes with a specified name. This name is printed to indicate that the chatbot is starting up.
2. Getting User Input (get_text_input method):
This method prompts the user to enter text input, and it returns the input provided by the user.
3. Text Output (text_to_speech method):
This method takes a text input and prints it to the console, simulating the chatbot's response.
4. Wake-Up Check (wake_up method):
This method checks if the chatbot's name (converted to lowercase) is mentioned in the text input. If it is, it returns True; otherwise, it returns False.
5. Time Action (action_time method):
This method retrieves the current time and returns it in the format "HH:MM".








In [4]:
# Building the ChatBot class
class ChatBot():
    def __init__(self, name):
        # Initialize the ChatBot with a name
        print("----- Starting up", name, "-----")
        self.name = name

    def get_text_input(self):
        # Method to get user input
        return input("Enter text: ")

    @staticmethod
    def text_to_speech(text):
        # Method to output text to console
        print("ChatBot --> ", text)

    def wake_up(self, text):
        # Method to check if the ChatBot's name is mentioned in the input
        return True if self.name in text.lower() else False

    @staticmethod
    def action_time():
        # Method to get the current time
        return datetime.datetime.now().time().strftime('%H:%M')

To run the chatbot, we need to initialize the chatbot, load the necessary components (tokenizer and conversational pipeline), greet the user, handle user input, process commands, engage in conversation, and exit gracefully when prompted. Here is how this happens:
1. **Initializing the ChatBot:** An instance of the ChatBot class is created with the name "ChatBot". This initializes the chatbot and prints a startup message indicating its name.
2. **Loading the Pre-trained Tokenizer:** The code loads a pre-trained tokenizer using transformers.AutoTokenizer.from_pretrained(). The tokenizer is loaded from the "microsoft/DialoGPT-medium" model, and the padding side is set to "left" to address a specific error.
3. **Creating the Conversational Pipeline:** A conversational pipeline is created using transformers.pipeline(). This pipeline utilizes the pre-trained DialoGPT-medium model for conversational generation and uses the loaded tokenizer for tokenization.
4. **Setting Environment Variable for Tokenization:** An environment variable "TOKENIZERS_PARALLELISM" is set to "true". This is done to enable parallel tokenization, which can improve performance.
5. **Greeting the User:** The chatbot greets the user with the message "Hello, I am ChatBot. How can I assist you today?" using the text_to_speech method.
6. **Main Loop:** The code enters a while loop (while ex:) where ex is initially set to True. This loop continues until ex is set to False.
7. **User Input Processing:** Inside the loop, the chatbot waits for user input using the get_text_input method.
8. **Command Handling:** The user input is checked for specific commands such as wake-up commands (mentioning the chatbot's name), time-related commands, polite responses, and exit commands. If any of these commands are detected, the chatbot responds accordingly.
9. **Conversation Continuation:** If the user input does not match any predefined commands, the chatbot engages in a conversation. It uses the pre-trained model to generate a response based on the input. The conversation history is updated with each exchange.
10. **Outputting Responses:** The chatbot's response is printed to the console using the text_to_speech method.
11. **Exiting the Loop:** If the user enters an exit command (e.g., "exit", "close", or "bye"), the loop exits, and the chatbot prints a closing message.
12. **Closing Message:** Finally, a closing message is printed indicating that the chatbot is shutting down.








In [ ]:
# Running the ChatBot
if __name__ == "__main__":
    # Initialize the ChatBot
    chatbot = ChatBot(name="ChatBot")

    # Load the pre-trained tokenizer
    tokenizer = transformers.AutoTokenizer.from_pretrained("microsoft/DialoGPT-medium", padding_side='left')  # Set padding side to 'left'

    # Load the pre-trained model
    model = transformers.AutoModelForCausalLM.from_pretrained("microsoft/DialoGPT-medium")

    # DEPRECATED: Create the conversational pipeline using the pre-trained model and tokenizer
    # nlp = transformers.pipeline("conversational", model="microsoft/DialoGPT-medium", tokenizer=tokenizer) # This line is replaced with the generate function below

    # Set environment variable for parallel tokenization
    os.environ["TOKENIZERS_PARALLELISM"] = "true"

    # Greet the user
    chatbot.text_to_speech("Hello, I am ChatBot. How can I assist you today?")

    # Main loop
    ex=True
    while ex:
        # Get user input
        text = chatbot.get_text_input()

        ## Check for wake up command
        if chatbot.wake_up(text) is True:
            # Respond if wake up command is detected
            res = "Hello, I am ChatBot. How can I assist you today?"

        ## Check for time command
        elif "time" in text:
            # Respond with the current time
            res = chatbot.action_time()

        ## Check for polite responses
        elif any(i in text for i in ["thank","thanks"]):
            # Respond politely
            res = np.random.choice(["You're welcome!", "Anytime!", "No problem!", "Cool!", "I'm here if you need me!", "Mention not"])

        ## Check for exit commands
        elif any(i in text for i in ["exit","close", "bye"]):
            # Respond to exit commands and exit the loop
            res = np.random.choice(["Tata", "Have a good day", "Bye", "Goodbye", "Hope to meet soon", "Peace out!"])
            ex=False

        ## Continue conversation
        else:
            # Start conversation with the ChatBot
            # conversation = nlp(transformers.Conversation(text), pad_token_id=50256) # This line is replaced with the generate function below
            new_user_input_ids = tokenizer.encode(text + tokenizer.eos_token, return_tensors='pt')

            # append the new user input tokens to the chat history
            bot_input_ids = new_user_input_ids

            # generate a response
            chat_history_ids = model.generate(bot_input_ids, max_length=1000, pad_token_id=tokenizer.eos_token_id)

            # Get the latest response from the ChatBot
            res = tokenizer.decode(chat_history_ids[:, bot_input_ids.shape[-1]:][0], skip_special_tokens=True)
            # Extract the ChatBot's response from the conversation history
            # res = res[res.find("bot >> ")+6:].strip() # This line is no longer needed

        # Output the response to the user
        chatbot.text_to_speech(res)

    # Print closing message
    print("----- Closing down ChatBot -----")

----- Starting up ChatBot -----


/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/614 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/1.04M [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/642 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/863M [00:00<?, ?B/s]

#**2. Chatbot Using OpenAI**
Yes, GPT!

Try the solution in the video out below. I've started you off below with installing the appropriate version of openai

In [ ]:
from IPython.display import IFrame  # This is just for me so I can embed videos
IFrame(src="https://www.youtube.com/embed/q5HiD5PNuck?si=ETpeM3-4NInvR_s9", width=560, height=315)

In [ ]:
#!pip install OpenAI
!pip install openai==0.28 # You will need this version of openai to make the code in the video work.

In [ ]:
import openai

Now get a project API key from https://platform.openai.com/api-keys and add it below

In [ ]:
openai.api_key = 'your API key here'

Type here the code from the video and try it out

#**3. A More Advanced Chatbot Using Chatterbot**
This example comes from [hubspot](https://blog.hubspot.com/website/python-ai-chat-bot). The code uses the [Chatterbot AI framework](https://chatterbot.readthedocs.io/en/stable/), a conversational dialog engine. For the Chatterbot Quickstart, check [here](https://chatterbot.readthedocs.io/en/stable/quickstart.html).

***NOTE:*** Implementing Chatterbot on Google Colab requires rolling back the Google Colab Python version to 3.8-full. There may be other version incompatibilities lurking around. I encourage you to try this out if you have time.

In [ ]:
# Cleaning up before rolling back Python version--run twice and ignore the warnings
!pip cache purge
!pip install --upgrade pip setuptools
!pip install ez_setup
!python --version # should show Python3.10.xx

In [ ]:
# Install an earlier version of Python (< 3.9) to get Chatterbot running
!sudo apt-get install python3.8-full
!sudo apt-get update -y
!sudo update-alternatives --install /usr/bin/python3 python3 /usr/bin/python3.8 1
!sudo update-alternatives --config python3
!python --version

In [ ]:
#!python3.8 -m pip install --upgrade pip
#!python --version
!sudo apt install python3-pip
#!wget https://bootstrap.pypa.io/get-pip.py
#!python3.8 get-pip.py

In [ ]:
## 1. Installing required libraries--this takes a looooooooong time!

#!pip install pyyaml==5.1.1 # should not be necessary
!pip install chatterbot
!python -m chatterbot --version
!pip install chatterbot-corpus

Now we are ready to import the libraries we need. Note that the code below is untested!

In [ ]:
## 2. Importing required libraries
import os

# import chatterbot
from chatterbot import ChatBot
from chatterbot.trainers import ListTrainer
from chatterbot.trainers import ChatterBotCorpusTrainer

In [ ]:
## 3. Create and name the chatbot
chatbot = ChatBot('MyChatBot')

In [ ]:
## 4. Training the chatbot with predefined corpus
trainer = ChatterBotCorpusTrainer(chatbot)
trainer.train("chatterbot.corpus.english")

In [ ]:
## 5. Testing the chatbot
response = chatbot.get_response("Hello, how are you?")
print(response)

In [ ]:
## 6. Training with Custom Data
from chatterbot.trainers import ListTrainer
trainer = ListTrainer(chatbot)
trainer.train([
"How are you?",
"I am good.",
"That is good to hear.",
"Thank you",
"You're welcome."
])

In [ ]:
## 7. Integrating chatbot into a webpage--this code has not been debugged.
from flask import Flask, render_template, request
app = Flask(__name__)
@app.route("/")
def home():
return render_template("index.html")
@app.route("/get")
def get_bot_response():
userText = request.args.get('msg')
return str(englishBot.get_response(userText))
if __name__ == "__main__":
app.run()

#4.**Challenges and Solutions in Building Python AI Chatbots**

1. **Challenge 1: Understanding User Intent**
   
   **Problem:** One of the biggest challenges in chatbot development is accurately understanding user intent. As language can be ambiguous and context-dependent, deciphering what a user truly means can be complex.

   **Solution:** Utilize NLP techniques like Named Entity Recognition (NER) and Intent Classification to interpret user input. Leverage machine learning models trained on large datasets to better recognize and respond to varied user queries.

2. **Challenge 2: Handling Conversational Context**
   **Problem:*** Maintaining the context of a conversation is crucial for delivering coherent responses. Without this, the chatbot might not understand references to previous messages, leading to a disjointed conversation.

   **Solution:** Implement context management in your chatbot using techniques like dialogue management and session tracking. Libraries like Rasa provide tools for managing conversational context.

3. **Challenge 3: Dealing with Unfamiliar Queries**
   **Problem:** Chatbots, especially rule-based ones, might stumble upon unfamiliar or out-of-scope queries, which can disrupt the user experience.

   **Solution:** Train your chatbot to handle unfamiliar queries gracefully. This could involve directing users to human support or suggesting alternate queries. Additionally, incorporate regular updates and training to your chatbot based on new and trending queries.

4. **Challenge 4: Lack of Personalization**
   **Problem:** Generic responses can make interactions with a chatbot feel mechanical and impersonal, diminishing user engagement.

   **Solution:** Implement personalization in your chatbot. This could range from using the user's name to tailoring responses based on user preferences and past interactions.

5. **Challenge 5: Scaling and Deployment**
   **Problem:** As your chatbot gets more complex and traffic increases, it may face issues related to performance, scalability, and deployment.

   **Solution:** Plan for scalability from the get-go. Utilize scalable cloud services and robust deployment practices. Monitor performance regularly and optimize as needed.

Remember, overcoming these challenges is part of the journey of developing a successful chatbot. Each challenge presents an opportunity to learn and improve, ultimately leading to a more sophisticated and engaging chatbot.